In [3]:
# Must contain this cell, in order to import from other folders
import os
import sys
sys.path.append(os.getcwd() + '/..')

%reload_ext autoreload
%autoreload 2

In [1]:
import pytorch_lightning as pl

# Set random seed
random_seed = 42
pl.seed_everything(random_seed)

# Create & fit the model using Pytorch Lightning's Trainer
trainer = pl.Trainer.from_argparse_args(args)
trainer.fit(model, datamodule=datamodule)

if args.fast_dev_run is None:
    trainer.test(datamodule=datamodule)

In [1]:
import torch.nn as nn

layer = nn.Linear(3, 2)

layer.in_features

3